### 수집할 지역
- 망원
- 합정
- 홍대
- 상수
- 연남
- 연희
- 신촌
- 이대
- 서교동

In [1]:
import requests
import pandas as pd
from dotenv import load_dotenv
load_dotenv('../05_data_scraping/.env')
import os
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import time
from konlpy.tag import Mecab
mecab = Mecab()
import re
from datetime import datetime

In [2]:
def string_delete(text) :
    return text.replace("<b>","").replace("</b>", "").replace("&quot;", "")

def parse_date(date_str):
    # 날짜 문자열을 datetime 객체로 변환
    return datetime.strptime(date_str, "%Y%m%d")

def make_df(df_name) :
    for i in df_name :
        for idx in range(0, len(df_name[i])) : 
            df_name[i][idx] = string_delete(df_name[i][idx])
    df_name['postdate'] = df_name['postdate'].apply(parse_date)
    return df_name

In [3]:
def w2v(keyword) :
    client_id = os.getenv('client_id') # 환경 변수에 접근
    client_pw = os.getenv('client_pw') # 환경 변수에 접근

    stopwords = pd.read_csv('https://raw.githubusercontent.com/haram4th/ablearn/main/%ED%95%9C%EA%B5%AD%EC%96%B4%EB%B6%88%EC%9A%A9%EC%96%B4100.txt', header = None)
    split_keyword = keyword.split(' ')
    ex_stop_words = [f'{split_keyword[0]}역']
    for i in ex_stop_words : 
        stopwords = stopwords.drop(stopwords[stopwords[0] == i].index)
    stopwords.reset_index(drop=True, inplace=True)
    
    url = "https://openapi.naver.com/v1/search/blog.json"
    final_data_keyword = pd.DataFrame()
    start = time.time() # 시작
    for i in range(1, 1000) :
        payload = {'query' : keyword, 'display' : 100, 'start' : i, 'sort' : 'sim'}
        headers = {'X-Naver-Client-Id' : client_id, 'X-Naver-Client-Secret' : client_pw}
        r = requests.get(url, params = payload, headers = headers)
        if r.status_code == 200 :
            data = r.json()
        else :
            print("Error Code:", r.status_code)
            
        df_keyword_sim = pd.json_normalize(data['items'])
        entire_data_keyword_sim = make_df(df_keyword_sim)
        final_data_keyword = pd.concat([final_data_keyword, entire_data_keyword_sim], ignore_index=True)
        # 형태소 분석 후 후처리
    def process_title(title, keyword):
        # '망리단길'을 임시 토큰으로 변경
        split_keyword = keyword.split(' ')
        title = title.replace(split_keyword[0], '우주')
        # 형태소 분석
        words = mecab.pos(re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", title))
        # 임시 토큰을 원래의 '망리단길'로 복원
        processed_words = [(word[0].replace('우주', keyword), word[1]) for word in words]
        return processed_words
        
    # DataFrame에 적용
    final_data_keyword['title2'] = final_data_keyword['title'].apply(lambda x : process_title(x, keyword))

    target = ['NNG', 'NNP', 'MAG', 'XR', 'NR', 'VV', 'VA', 'VCP', 'VCN', 'MM', 'IC', 'XPN', 'UNKNOWN']

    # 형태소에서 특정 단어 제거 및 태그 필터링
    def extract_words(words_list) : 
        return [word for word, pos in words_list if pos in target]
    final_data_keyword['filtered'] = final_data_keyword['title2'].apply(lambda x : extract_words(x))
    final_data_keyword['filtered_list'] = final_data_keyword['filtered'].apply(lambda x : eval(x) if isinstance(x, str) else x)
    from gensim.models import Word2Vec
    model_keyword_pos = Word2Vec(final_data_keyword['filtered_list'], vector_size=100, window=5, min_count=2, workers=8, sg=0)
    model_keyword_pos_result = model_keyword_pos.wv.most_similar(keyword, topn = 50)
    # 워드투벡터 결과 소수점 4자리로 변경하기
    w2v_result = list((item[0], round(item[1], 4)) for item in model_keyword_pos_result)
    print(f"{time.time()-start:.4f} sec")
    return keyword, w2v_result

In [5]:
mangwon = w2v('망원')
mangwon

/home/user/miniforge3/envs/project/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


280.7914 sec


('망원',
 [('그', 0.3156),
  ('서울', 0.3138),
  ('길', 0.301),
  ('시장', 0.3009),
  ('근처', 0.2919),
  ('데이트', 0.284),
  ('베르', 0.2812),
  ('맛집', 0.2807),
  ('단', 0.2732),
  ('분위기', 0.2707),
  ('좋', 0.2637),
  ('대형', 0.2569),
  ('웨이즈', 0.2552),
  ('이색', 0.2541),
  ('추천', 0.2521),
  ('부채살', 0.2498),
  ('동', 0.2484),
  ('어거스트', 0.2463),
  ('디어', 0.2459),
  ('당도', 0.2391),
  ('햇살', 0.2329),
  ('큐', 0.2282),
  ('아늑', 0.2274),
  ('메뉴', 0.2266),
  ('코너', 0.2243),
  ('썸', 0.224),
  ('망리', 0.223),
  ('함바', 0.2223),
  ('모녀', 0.2216),
  ('스터스', 0.2207),
  ('느낌', 0.2165),
  ('가', 0.2134),
  ('내산', 0.2132),
  ('오랜만', 0.2076),
  ('돈', 0.2062),
  ('금방', 0.2044),
  ('젤라', 0.2031),
  ('포레스트', 0.1993),
  ('동주', 0.1992),
  ('하늘', 0.1977),
  ('당일', 0.1965),
  ('나들이', 0.1961),
  ('프리', 0.1952),
  ('민', 0.1936),
  ('산장', 0.1928),
  ('띵', 0.192),
  ('차', 0.189),
  ('후기', 0.1884),
  ('네임', 0.1883),
  ('휴식', 0.1853)])

In [6]:
hapjeong = w2v('합정')
hapjeong

286.6396 sec


('합정',
 [('추천', 0.3942),
  ('찾', 0.3178),
  ('분위기', 0.3072),
  ('이자', 0.304),
  ('술잔', 0.2956),
  ('데이트', 0.2895),
  ('소란', 0.2859),
  ('조용', 0.2837),
  ('독서', 0.2812),
  ('도란', 0.277),
  ('가성', 0.2736),
  ('감자전', 0.272),
  ('감성', 0.269),
  ('망원', 0.268),
  ('방문', 0.2612),
  ('코드', 0.2568),
  ('좋', 0.2539),
  ('서재', 0.248),
  ('피온', 0.2471),
  ('포장마차', 0.2421),
  ('카', 0.2365),
  ('맛집', 0.2365),
  ('피맥', 0.2361),
  ('서울', 0.2353),
  ('버', 0.2351),
  ('부위', 0.2326),
  ('하이볼', 0.231),
  ('루', 0.2287),
  ('근처', 0.2259),
  ('참', 0.2235),
  ('코스', 0.2231),
  ('트릴', 0.2229),
  ('살롱', 0.2179),
  ('전문점', 0.2178),
  ('토리', 0.2147),
  ('부근', 0.2125),
  ('상수', 0.2108),
  ('의사', 0.2098),
  ('타운', 0.2085),
  ('디', 0.2084),
  ('아소토', 0.2073),
  ('대박', 0.2022),
  ('오', 0.2006),
  ('홍대', 0.1982),
  ('떠나', 0.1963),
  ('규', 0.1926),
  ('공명', 0.1923),
  ('합정역', 0.1917),
  ('발리', 0.1908),
  ('특수', 0.1902)])

In [7]:
hongdae = w2v('홍대')
hongdae

305.7709 sec


('홍대',
 [('서울', 0.3492),
  ('입구', 0.3051),
  ('연남', 0.2765),
  ('추천', 0.2746),
  ('뷰티', 0.2739),
  ('좋', 0.2637),
  ('플', 0.2595),
  ('신상', 0.25),
  ('프리모', 0.2469),
  ('삭면', 0.2343),
  ('언더', 0.232),
  ('같', 0.2298),
  ('홍대역', 0.2281),
  ('핫', 0.2278),
  ('갈비찜', 0.2253),
  ('시도', 0.2226),
  ('페이', 0.2199),
  ('브런치', 0.2195),
  ('동교동', 0.2182),
  ('집', 0.2179),
  ('가산', 0.2155),
  ('다시', 0.2145),
  ('트릴로', 0.2133),
  ('감성', 0.2119),
  ('프레', 0.2118),
  ('가성', 0.2078),
  ('소', 0.2018),
  ('짚신', 0.2012),
  ('백해', 0.1989),
  ('에스프레소', 0.1965),
  ('몬스터', 0.1956),
  ('가게', 0.1947),
  ('수역', 0.193),
  ('대호', 0.191),
  ('연남동', 0.1905),
  ('야키', 0.1902),
  ('바치', 0.1886),
  ('바나', 0.1885),
  ('상수역', 0.1879),
  ('대', 0.1873),
  ('교동', 0.1872),
  ('이색', 0.1871),
  ('포케', 0.1866),
  ('세계', 0.1847),
  ('빠', 0.1843),
  ('영화', 0.184),
  ('들풀', 0.1838),
  ('배터리', 0.1806),
  ('피프', 0.1792),
  ('철길', 0.179)])

In [8]:
sangsoo = w2v('상수')
sangsoo

296.5271 sec


('상수',
 [('홍대', 0.3683),
  ('추천', 0.3056),
  ('분위기', 0.2895),
  ('조용', 0.2811),
  ('근처', 0.2596),
  ('서울', 0.2568),
  ('맛집', 0.2511),
  ('천장', 0.2485),
  ('주도', 0.241),
  ('딱', 0.2311),
  ('데이트', 0.2282),
  ('상수역', 0.2266),
  ('인천', 0.2244),
  ('전환', 0.2242),
  ('미식가', 0.2229),
  ('좋', 0.2149),
  ('인형', 0.2122),
  ('유라', 0.2117),
  ('피자', 0.2093),
  ('눈', 0.2073),
  ('집', 0.2062),
  ('거리', 0.2051),
  ('성비', 0.2042),
  ('독특', 0.2042),
  ('후기', 0.2035),
  ('피', 0.2001),
  ('립스', 0.1985),
  ('소문', 0.1983),
  ('팥빙수', 0.1981),
  ('맛있', 0.198),
  ('내산', 0.1917),
  ('퍼', 0.1911),
  ('갑', 0.1892),
  ('널찍', 0.1891),
  ('심심', 0.188),
  ('밸런스', 0.1873),
  ('소개', 0.187),
  ('무제', 0.1868),
  ('즐기', 0.1862),
  ('포장', 0.1843),
  ('테라스', 0.1841),
  ('내리', 0.1838),
  ('찾', 0.1838),
  ('디', 0.1838),
  ('많', 0.1831),
  ('핫', 0.1809),
  ('돈', 0.1808),
  ('제주', 0.1785),
  ('트라움', 0.1784),
  ('벙크', 0.1781)])

In [9]:
yeonnam = w2v('연남')
yeonnam

269.7600 sec


('연남',
 [('동', 0.5464),
  ('추천', 0.5109),
  ('홍대', 0.4541),
  ('맛집', 0.3883),
  ('좋', 0.3337),
  ('분위기', 0.3322),
  ('가성', 0.2921),
  ('조용히', 0.2676),
  ('파스타', 0.2638),
  ('돈', 0.2594),
  ('종로', 0.2593),
  ('앤티크', 0.2531),
  ('옫', 0.2496),
  ('서울', 0.2443),
  ('한옥', 0.2437),
  ('이', 0.2397),
  ('후기', 0.2396),
  ('경양식', 0.2311),
  ('장소', 0.2292),
  ('핫', 0.2282),
  ('플', 0.2278),
  ('발견', 0.2276),
  ('필수', 0.2271),
  ('음식점', 0.2259),
  ('홍대입구역', 0.2254),
  ('후', 0.2248),
  ('완벽', 0.2229),
  ('흠뻑', 0.2178),
  ('데이트', 0.2162),
  ('하이볼', 0.2132),
  ('모두', 0.2125),
  ('성비', 0.2115),
  ('밥집', 0.2108),
  ('맛있', 0.2067),
  ('안', 0.2057),
  ('코스', 0.2047),
  ('오', 0.2035),
  ('파', 0.2013),
  ('재', 0.1996),
  ('다', 0.1995),
  ('베이커', 0.1987),
  ('덮밥', 0.1931),
  ('기집', 0.1911),
  ('점심', 0.1884),
  ('찾', 0.1883),
  ('감성', 0.188),
  ('신상', 0.1877),
  ('집', 0.1866),
  ('딱', 0.1856),
  ('정말', 0.1837)])

In [10]:
yeonhui = w2v('연희')
yeonhui

264.6912 sec


('연희',
 [('동', 0.3029),
  ('잔여', 0.271),
  ('있', 0.2703),
  ('에디션', 0.2476),
  ('코다', 0.2424),
  ('절차', 0.2329),
  ('정보', 0.2315),
  ('강조', 0.2207),
  ('전매', 0.2182),
  ('조건', 0.214),
  ('사항', 0.2137),
  ('쏘', 0.2128),
  ('소개', 0.2128),
  ('내용', 0.2126),
  ('찾', 0.2113),
  ('단골', 0.2104),
  ('분식', 0.2072),
  ('청라', 0.2054),
  ('즐기', 0.2029),
  ('삼학', 0.2015),
  ('리필', 0.201),
  ('디', 0.2004),
  ('안내', 0.2003),
  ('넘치', 0.1977),
  ('아파트', 0.1974),
  ('만나', 0.1968),
  ('튀김', 0.1954),
  ('웨이팅', 0.1932),
  ('세대', 0.1918),
  ('파크', 0.1903),
  ('박나래', 0.1895),
  ('밋', 0.1884),
  ('방생', 0.187),
  ('효자동', 0.1867),
  ('모든', 0.1852),
  ('해산물', 0.1848),
  ('에버그린', 0.1845),
  ('진수', 0.1831),
  ('향연', 0.1823),
  ('가득', 0.1819),
  ('쿳사연희', 0.1815),
  ('억만', 0.179),
  ('산책로', 0.1786),
  ('세부', 0.1785),
  ('부동산', 0.1776),
  ('가족', 0.1761),
  ('인천', 0.1751),
  ('메뉴', 0.1744),
  ('보물', 0.1737),
  ('분양', 0.1734)])

In [13]:
yeonhui_dong = w2v('연희동')
yeonhui_dong

279.4044 sec


('연희동',
 [('뜨개실', 0.3096),
  ('바보', 0.2931),
  ('오프라인', 0.2883),
  ('비', 0.2709),
  ('일정', 0.2662),
  ('분위기', 0.255),
  ('양', 0.2487),
  ('낙낙', 0.2399),
  ('와인', 0.234),
  ('현황', 0.2321),
  ('공원', 0.2283),
  ('갈비', 0.22),
  ('부상', 0.2108),
  ('자세', 0.2108),
  ('최저', 0.2075),
  ('조용', 0.207),
  ('추천', 0.2068),
  ('갈비탕', 0.2062),
  ('곳', 0.2056),
  ('알', 0.2029),
  ('단체', 0.2025),
  ('라테', 0.2018),
  ('마음대로', 0.2012),
  ('연세대', 0.2003),
  ('다도', 0.1982),
  ('전원', 0.1972),
  ('차이나타운', 0.1954),
  ('점', 0.1949),
  ('유명', 0.1945),
  ('힙', 0.1927),
  ('혼', 0.1912),
  ('탐방기', 0.1899),
  ('까페', 0.1895),
  ('회식', 0.189),
  ('ㅣ', 0.1888),
  ('통창', 0.1886),
  ('연남동', 0.1871),
  ('데이트', 0.1866),
  ('프로토콜', 0.1843),
  ('가능', 0.1836),
  ('굴비', 0.1822),
  ('무스', 0.1807),
  ('호주', 0.1804),
  ('정찬', 0.1788),
  ('자리', 0.1779),
  ('수업', 0.1768),
  ('턴', 0.1754),
  ('스테이크', 0.1753),
  ('들살', 0.1747),
  ('고소', 0.1747)])

In [14]:
yeonhui_dong_seoul = w2v('연희동 서울')
yeonhui_dong_seoul

280.4595 sec


('연희동 서울',
 [('통째', 0.3184),
  ('현상', 0.2959),
  ('분위기', 0.266),
  ('빙수', 0.2593),
  ('바이인', 0.2561),
  ('팬케이크', 0.2524),
  ('살', 0.2467),
  ('애정', 0.2429),
  ('한적', 0.2364),
  ('재미', 0.2281),
  ('가능', 0.2237),
  ('상대', 0.218),
  ('시즌', 0.2148),
  ('쉬', 0.2127),
  ('식단', 0.2123),
  ('추천', 0.2105),
  ('대형', 0.2092),
  ('그로', 0.204),
  ('넓', 0.2001),
  ('종', 0.1985),
  ('라울', 0.1983),
  ('연세대', 0.1937),
  ('어스', 0.1928),
  ('발견', 0.1912),
  ('인생', 0.1899),
  ('조용', 0.1893),
  ('우디', 0.1841),
  ('산도', 0.184),
  ('코스', 0.1837),
  ('새', 0.1834),
  ('나만', 0.1833),
  ('주차', 0.1827),
  ('좋', 0.1817),
  ('후기', 0.1808),
  ('테라스', 0.1801),
  ('다이어트', 0.1784),
  ('개조', 0.1767),
  ('화이트', 0.1757),
  ('뷰', 0.1748),
  ('연희동 서울인', 0.1737),
  ('탑', 0.1719),
  ('비', 0.1711),
  ('브륄', 0.1709),
  ('내산', 0.17),
  ('점', 0.17),
  ('맛집', 0.1672),
  ('아메리카노', 0.1671),
  ('보', 0.1666),
  ('줄서', 0.1656),
  ('쿠치', 0.1655)])

In [15]:
seoul_yeonhui_dong = w2v('서울 연희동')
seoul_yeonhui_dong

277.5368 sec


('서울 연희동',
 [('추천', 0.2518),
  ('분위기', 0.2318),
  ('한적', 0.2213),
  ('대형', 0.2154),
  ('인생', 0.21),
  ('배달', 0.1955),
  ('급매', 0.1953),
  ('필라테스', 0.1892),
  ('연희동', 0.1824),
  ('주차', 0.1791),
  ('밥집', 0.178),
  ('아메리카노', 0.1777),
  ('우디', 0.1687),
  ('넓', 0.1658),
  ('아정', 0.1633),
  ('에스프레소', 0.1623),
  ('브런치', 0.1621),
  ('필터', 0.1619),
  ('시즌', 0.1612),
  ('북카페', 0.1607),
  ('쓰리', 0.1566),
  ('점', 0.1539),
  ('연예인', 0.1535),
  ('보', 0.1525),
  ('야드', 0.1507),
  ('가능', 0.1499),
  ('새', 0.1481),
  ('신상', 0.1468),
  ('드론', 0.1457),
  ('연세대', 0.1445),
  ('연남동', 0.1433),
  ('오븐', 0.1432),
  ('별밥', 0.1432),
  ('조용', 0.1425),
  ('뷰', 0.1411),
  ('풀코스', 0.1381),
  ('돈', 0.1379),
  ('베', 0.1374),
  ('데스', 0.1339),
  ('직접', 0.1335),
  ('거트', 0.1332),
  ('노트북', 0.1315),
  ('ㅣ', 0.1309),
  ('단아', 0.1305),
  ('쿠치', 0.13),
  ('하스', 0.1293),
  ('한식', 0.1288),
  ('맛있', 0.1284),
  ('있', 0.1284),
  ('덤', 0.1281)])

In [11]:
sinchon = w2v('신촌')
sinchon

285.6283 sec


('신촌',
 [('스키야키', 0.2786),
  ('잼', 0.275),
  ('츠', 0.272),
  ('현지', 0.2701),
  ('근처', 0.2687),
  ('이색', 0.2607),
  ('모듬', 0.2606),
  ('훈', 0.2578),
  ('신촌역', 0.2568),
  ('초밥', 0.2545),
  ('간절히', 0.251),
  ('초', 0.2509),
  ('데이트', 0.2466),
  ('스', 0.245),
  ('탐방', 0.2427),
  ('딤섬', 0.2423),
  ('가성', 0.2378),
  ('계란', 0.2314),
  ('케', 0.2301),
  ('스릴러', 0.2289),
  ('후기', 0.2248),
  ('도리탕', 0.2234),
  ('현백', 0.2231),
  ('앞', 0.2228),
  ('긴자', 0.2222),
  ('일식', 0.2201),
  ('오', 0.2171),
  ('치히로', 0.2157),
  ('곱', 0.215),
  ('폭포', 0.2129),
  ('페샬나잇트', 0.2123),
  ('현대', 0.2121),
  ('딤', 0.2119),
  ('도쿠', 0.2107),
  ('최고', 0.2092),
  ('라라', 0.2079),
  ('업', 0.2061),
  ('스케이프', 0.2032),
  ('탕화', 0.202),
  ('좋', 0.2014),
  ('꾸', 0.2014),
  ('템', 0.2),
  ('쿄', 0.1966),
  ('유야', 0.1963),
  ('비갑', 0.1951),
  ('사발', 0.1936),
  ('화원', 0.1931),
  ('밥', 0.193),
  ('가', 0.1915),
  ('샐러드', 0.1886)])

In [12]:
edae = w2v('이대')
edae

275.3665 sec


('이대',
 [('푸딩', 0.4113),
  ('딩', 0.3174),
  ('베지', 0.3035),
  ('거트', 0.3026),
  ('신촌', 0.2932),
  ('반년', 0.2837),
  ('푸푸', 0.2722),
  ('식물', 0.2717),
  ('장어', 0.2695),
  ('바로우', 0.2625),
  ('다른', 0.259),
  ('브런치', 0.2586),
  ('연보라', 0.2576),
  ('있', 0.2572),
  ('천', 0.2508),
  ('소정', 0.2433),
  ('돌입', 0.2428),
  ('차원', 0.2393),
  ('삭면', 0.2362),
  ('부', 0.2336),
  ('부드럽', 0.2315),
  ('신부', 0.2301),
  ('다이어트', 0.2264),
  ('기원', 0.2262),
  ('란스', 0.2246),
  ('맞이', 0.2228),
  ('캐빈', 0.2223),
  ('톳', 0.2222),
  ('스콘', 0.2217),
  ('편안', 0.2214),
  ('마마', 0.221),
  ('캘리스', 0.2207),
  ('무스', 0.2182),
  ('미역국', 0.218),
  ('세브', 0.2172),
  ('받', 0.2154),
  ('식단', 0.2105),
  ('전문점', 0.2094),
  ('로스팅', 0.2089),
  ('추천', 0.2071),
  ('후기', 0.2068),
  ('블루리본', 0.2064),
  ('채림', 0.2032),
  ('만드', 0.2027),
  ('치', 0.2004),
  ('고치', 0.1995),
  ('키친', 0.1992),
  ('보돌', 0.1991),
  ('스테이크', 0.1985),
  ('붓', 0.1983)])

In [4]:
seogyo = w2v('서교동')
seogyo

/home/user/miniforge3/envs/project/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


271.9233 sec


('서교동',
 [('홍대', 0.2954),
  ('강추', 0.2911),
  ('분위기', 0.2722),
  ('가로수길', 0.2474),
  ('역점', 0.2434),
  ('포스팅', 0.2392),
  ('결사', 0.2333),
  ('밥술', 0.2312),
  ('세상', 0.2261),
  ('차단', 0.225),
  ('카다멈', 0.2172),
  ('좋', 0.2147),
  ('대서리', 0.2135),
  ('학원', 0.2079),
  ('푸딩', 0.2076),
  ('특별', 0.207),
  ('유원재', 0.2058),
  ('마카롱', 0.2012),
  ('최적화', 0.2008),
  ('해주', 0.199),
  ('반상', 0.1987),
  ('우스', 0.1983),
  ('곧', 0.1977),
  ('속', 0.1942),
  ('맛과', 0.1918),
  ('형제', 0.1898),
  ('천정', 0.189),
  ('동파육', 0.1876),
  ('합정역', 0.1873),
  ('로드', 0.1872),
  ('지정', 0.1853),
  ('안심', 0.1821),
  ('이양', 0.1814),
  ('방어', 0.1812),
  ('터치', 0.1808),
  ('국밥', 0.1783),
  ('누하', 0.1757),
  ('다수', 0.1729),
  ('아일라', 0.1727),
  ('점', 0.1701),
  ('마라', 0.1692),
  ('상태', 0.1686),
  ('민', 0.1685),
  ('끝', 0.168),
  ('저렴', 0.1668),
  ('있', 0.1636),
  ('성비', 0.1635),
  ('물티슈', 0.1626),
  ('캠핑', 0.1623),
  ('닭발', 0.1622)])